# EM Waves in Matter

In this tutorial, we will use the open-source electromagnetic simulation software MEEP (MIT Electromagnetic Equation Propagation), a finite difference time domain Maxwell solver to simulate the propagation of EM waves through matter, and plasma in particular. To make things easier, we will use the PlasMEEP library, which is effectively a wrapper for MEEP that makes it easier to build plasma-themed electromagnetic devices.

Let's begin by importing the relevant libraries and defining the simulation parameters.


In [ ]:
from plasmeep.lib import Plasmeep as pm
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 1000
import matplotlib.pyplot as plt
import numpy as np
import os

a = 0.01 #Dimensionalized unit length in meters
res = 50 #Simulation resolution
nx = 10  #x-dimension in a units
ny = 8   #y-dimension in a units
dpml = 1 #Thickness of perfectly matched layer (functions like a radiative BC)
B = np.array([0,0,0])    #Externally applied magnetic field

### Propagation through glass (Snell's Law)

In an isotropic linear dielectric material, the dielectric constant is best-understood as being equivalent to the index of refraction squared. To illustrate this, for our first simulation we will send a plane wave into a glass prism at different angles to see how the fields behave. Feel free to adjust the angle of the prism, number of frames, or resolution in the cell above in case the simulation is running slowly on your machine.


In [ ]:
output = os.getcwd()+'/../outputs/plots/'
sim = pm(a, res, dpml, nx, ny, B = B) #Initialize PlasMEEP object

## Build Geometry
n = 1.52        #Index of refraction of glass
eps = n**2      #Relative permittivity
theta = np.pi/6 #Angle from normal incidence of the glass prism   

## Here we are calculating the position of the vertices of the glass prism
v1 = np.array([-2,-4,0])
if theta <= np.arctan(7/8):
    v2 = np.array([-2+8*np.tan(theta),4,0])
    v3 = np.array([5,4,0])
    v4 = np.array([5,-4,0])
    sim.Add_Prism(np.array([v1, v2, v3, v4]), eps = eps)
else:
    v2 = np.array([5,7*np.tan(np.pi/2-theta),0])
    v3 = np.array([5,-4,0])
    sim.Add_Prism(np.array([v1, v2, v3]), eps = eps)

## Now add source and simulate
freq = 0.8         #Non-dimensionalized frequency
pol = [0,0,1]      #Field polarization (in this case, E is pointing out of the screen)
src_loc = [-4,0,0] #Source center location
src_size = [0,8,0] #Source size (spans whole domain in y-direction here)
sim.Add_Cont_Source(freq, pol, src_loc, src_size)

sim.Viz_Domain(output+'/Glass_Prism_pi_6')
sim.Sim_And_Plot(1.0, 'E', 20, output, 'ez', out_prefix = 'Glass_Prism_pi_6',\
                 remove = True)
#sim.Sim_And_Plot_Gif(1.0, 'E', 20, output, 'ez', out_prefix = 'Glass_Prism_pi_6',\
#                     remove = True, filetype = 'gif')

### Higher dielectric constants compress the fields even more

We see from the results above that in addition to the refraction at off-normal incidence, the wavelength within the material is smaller than in free space, and the velocity of the wave is greatly reduced as well. For some materials like ceramics, the dielectric constant can be as high as 12.

In [ ]:
sim = pm(a, res, dpml, nx, ny, B = B) #Re-Initialize PlasMEEP object

## Build Geometry (simple for normal incidence)
sim.Add_Block(low_left = [-2,-4,0], size = [7,8,0], eps = 12)

## Now add source and simulate
freq = 0.5         #Non-dimensionalized frequency
pol = [0,0,1]      #Field polarization (in this case, E is pointing out of the screen)
src_loc = [-4,0,0] #Source center location
src_size = [0,8,0] #Source size (spans whole domain in y-direction here)
sim.Add_Cont_Source(freq, pol, src_loc, src_size)

sim.Viz_Domain(output+'/Ceramic_Prism')
sim.Sim_And_Plot(1.0, 'E', 20, output, 'ez', out_prefix = 'Ceramic_Prism',\
                 remove = True)
#sim.Sim_And_Plot_Gif(1.0, 'E', 20, output, 'ez', out_prefix = 'Ceramic_Prism',\
#                     remove = True, filetype = 'gif')

### Plasma allows us to access permittivity values less than 1
When the incident wave frequency is greater than the plasma frequency, the relative permittivity of the plasma is greater than 0 and less than 1. This causes waves to stretch -  and their phase velocity becomes superluminar.

In [ ]:
sim = pm(a, res, dpml, nx, ny, B = B) #Re-Initialize PlasMEEP object

## Build Geometry (simple for normal incidence)
# Note that we include a plasma frequency in the arguments of the function - 
# this designates that the block is made of plasma
sim.Add_Block(low_left = [-2,-4,0], size = [7,8,0], eps = 1, wp = 0.7, gamma = 0)
    
## Now add source and simulate
freq = 0.8         #Non-dimensionalized frequency
pol = [0,0,1]      #Field polarization (in this case, E is pointing out of the screen)
src_loc = [-4,0,0] #Source center location
src_size = [0,8,0] #Source size (spans whole domain in y-direction here)
sim.Add_Cont_Source(freq, pol, src_loc, src_size)

sim.Viz_Domain(output+'/Plasma_underdense')
sim.Sim_And_Plot(1.0, 'E', 20, output, 'ez', out_prefix = 'Plasma_underdense',\
                 remove = True)
#sim.Sim_And_Plot_Gif(1.0, 'E', 20, output, 'ez', out_prefix = 'Plasma_underdense',\
#                     remove = True, filetype = 'gif')

### We can access negative permittivities when fp > f
Here we can see that the plasma becomes reflective and enters the metallic regime.

In [ ]:
sim = pm(a, res, dpml, nx, ny, B = B) #Re-Initialize PlasMEEP object

## Build Geometry (simple for normal incidence)
sim.Add_Block(low_left = [-2,-4,0], size = [7,8,0], eps = 1, wp = 1.6, gamma = 0)
    
## Now add source and simulate
freq = 0.8         #Non-dimensionalized frequency
pol = [0,0,1]      #Field polarization (in this case, E is pointing out of the screen)
src_loc = [-4,0,0] #Source center location
src_size = [0,8,0] #Source size (spans whole domain in y-direction here)
sim.Add_Cont_Source(freq, pol, src_loc, src_size)

sim.Viz_Domain(output+'/Plasma_overdense')
sim.Sim_And_Plot(1.0, 'E', 20, output, 'ez', out_prefix = 'Plasma_overdense',\
                 remove = True)
#sim.Sim_And_Plot_Gif(1.0, 'E', 20, output, 'ez', out_prefix = 'Plasma_overdense',\
#                     remove = True, filetype = 'gif')

### What happens if fp = f_src?

In [ ]:
nx = 20 #Make domain longer to show the desired effect
sim = pm(a, res, dpml, nx, ny, B = B) #Re-Initialize PlasMEEP object

## Build Geometry (simple for normal incidence)
sim.Add_Block(low_left = [-7,-4,0], size = [17,8,0], eps = 1, wp = 0.5, gamma = 0)
    
## Now add source and simulate
freq = 0.5         #Non-dimensionalized frequency
pol = [0,0,1]      #Field polarization (in this case, E is pointing out of the screen)
src_loc = [-9,0,0] #Source center location
src_size = [0,8,0] #Source size (spans whole domain in y-direction here)
sim.Add_Cont_Source(freq, pol, src_loc, src_size)

sim.Viz_Domain(output+'/Plasma_ZIM')
sim.Sim_And_Plot(1.0, 'E', 20, output, 'ez', out_prefix = 'Plasma_ZIM',\
                 remove = True)
#sim.Sim_And_Plot_Gif(1.0, 'E', 20, output, 'ez', out_prefix = 'Plasma_ZIM',\
#                     remove = True, filetype = 'gif')

### There are ways to make a medium behave like a true zero-index medium
By using photonic crystal structures with exotic EM properties, we can get an array of dielectric rods to behave like a true zero-index medium. To get an idea of what a ZIM really behaves like and see the kinds of crazy things you can simulate with Meep, we'll construct such a device here. See 

*X. Huang, Y. Lai, Z. H. Hang, H. Zheng, and C. T. Chan, "Dirac cones induced by accidental degeneracy in photonic crystals and zero-refractive-index materials" Nat. Mater. 10, 582 (2011).* 

*J. A. Rodríguez, B. Wang, and M. A. Cappelli, "Dual-polarization Dirac cones in a simple 2D square lattice photonic crystal," Opt. Lett. 45, 2486-2489 (2020)* 

for some detailed discussion of devices like this. 

In [ ]:
## The domain needs to be much larger this time (this one may take a while)
res = 16
ny = 40
nx = 40
sim = pm(a, res, dpml, nx, ny, B = B) #Re-Initialize PlasMEEP object

## Build Geometry, this time an array of cylindrical rods
rod_eps = np.zeros((20,20,3))
rod_eps[:,:,0] = 12.5*np.ones((20,20))
sim.Rod_Array(r = 0.2, xyz_start = [-10, -10, 0], a_l = 1,\
              rod_eps = rod_eps, axis = [0, 0, 1])
    
## Now add source and simulate
freq = 0.541        #Non-dimensionalized frequency
pol = [0,0,1]       #Field polarization (in this case, E is pointing out of the screen)
src_loc = [-19,0,0] #Source center location
src_size = [0,20,0] #Source size (spans crystal in y-direction here)
sim.Add_Cont_Source(freq, pol, src_loc, src_size)

sim.Viz_Domain(output+'/Dirac_ZIM')
sim.Sim_And_Plot(1.0, 'E', 150, output, 'ez', out_prefix = 'Dirac_ZIM',\
                 remove = True)
#sim.Sim_And_Plot_Gif(1.0, 'E', 150, output, 'ez', out_prefix = 'Dirac_ZIM',\
#                     remove = True, filetype = 'gif', intensity = 1.5)